In [1]:
import pandas as pd
from neo4j import GraphDatabase
import requests
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j', 'neo4j'))

# BISOGNA FARE UNA COSA 4 STAGE:
<ol>
    <li>usare i nomi degli autori presi dall'url dell'immagine e vedere di recuperare quelli non presenti in artgraph;</li>
    <li>ci saranno ancora altri lavori che non vengono presi. Proviamo a prendere il nome dell'autore da <code>artwork_info</code> e di rifare l'operazione
    <li>ci saranno ancora altri lavori che non vengono presi. Se l'autore riscontra delle opere via API, vuol dire che bisogna passare a prendere le info puntuali dell'artwork, quindi prendere il campo <code>url</code> e, infine, provare il matching;
    <li>in ultimo, se l'autore non dà alcun risultato, è molto probabile che ci sia un <b>typo</b> . Va riconosciuto, modificato il tutto e rilanciato
</ol>

Get artworks information

In [4]:
artwork_info = pd.read_csv('WikiArt-Emotions/WikiArt-info.tsv', delimiter='\t')

,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie
...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin


Get name for trying to do a mapping with ArtGraph

In [5]:
artwork_info['name'] = artwork_info['Image URL'].map(lambda x: '_'.join(x.split('/')[-2:]))

,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL,name
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...,charles-courtney-curran_in-the-luxembourg-gard...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring,keith-haring_the-marriage-of-heaven-and-hell-1...
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai,j-zsef-rippl-r-nai_uncle-piacsek-in-front-of-t...
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller,vadym-meller_monk-for-the-play-mazeppa-1920.jpg
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie,david-wilkie_the-defence-of-sarago-a.jpg
...,...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger,rupprecht-geiger_oe-260-1957.jpg
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez,oscar-dominguez_m-quina-de-coser-electro-sexua...
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz,georg-baselitz_female-nude-on-a-kitchen-chair-...
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin,marie-laurencin_apollinaire-and-his-friends-19...


Function that checks if a painting is present in ArtGraph

In [6]:
def is_in_artgraph(param, value, driver):
    base_query = f'match (a:Artwork) where a.{param} = "{value}" return count(distinct a) as num'
    with driver.session(database='neo4j') as session:
        ans = session.run(base_query).data()
    return ans[0]['num']

In [7]:
artwork_info['name_in_artgraph'] = artwork_info.progress_apply(lambda x: is_in_artgraph('name', x['name'], driver), axis=1)

100%|██████████| 4119/4119 [07:25<00:00,  9.24it/s]


Taking artworks that are not present in artgraph

In [10]:
unretrieved = artwork_info[artwork_info.name_in_artgraph == 0]

,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL,name,name_in_artgraph
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai,j-zsef-rippl-r-nai_uncle-piacsek-in-front-of-t...,0
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller,vadym-meller_monk-for-the-play-mazeppa-1920.jpg,0
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie,david-wilkie_the-defence-of-sarago-a.jpg,0
8,57728255edc2cb3880f7644a,Minimalism,Robert Smithson,Mirror and Crushed Shells,1969,https://uploads7.wikiart.org/images/robert-smi...,https://www.wikiart.org/en/robert-smithson/mir...,https://www.wikiart.org/en/robert-smithson,robert-smithson_mirror-and-crushed-shells-1969...,0
9,5772843bedc2cb3880fd334d,Minimalism,Richard Serra,One Ton Prop (House of Cards),1969,https://uploads7.wikiart.org/images/richard-se...,https://www.wikiart.org/en/richard-serra/one-t...,https://www.wikiart.org/en/richard-serra,richard-serra_one-ton-prop-house-of-cards-1969...,0
...,...,...,...,...,...,...,...,...,...,...
4105,57728417edc2cb3880fca897,"Abstract Art,Cubism",David Bomberg,In the Hold,1914,https://uploads0.wikiart.org/images/david-bomb...,https://www.wikiart.org/en/david-bomberg/in-th...,https://www.wikiart.org/en/david-bomberg,david-bomberg_in-the-hold-1914.jpg,0
4108,57728752edc2cb388006d59e,Lyrical Abstraction,Stephen Mueller,Allstar Galilee,1970,https://uploads6.wikiart.org/images/stephen-mu...,https://www.wikiart.org/en/stephen-mueller/all...,https://www.wikiart.org/en/stephen-mueller,stephen-mueller_allstar-galilee-1970.jpg,0
4109,577286caedc2cb388005687d,Rococo,Johann Georg Pinzel,Samson tearing the lion&#39;s mouth,1760,https://uploads5.wikiart.org/images/johann-geo...,https://www.wikiart.org/en/johann-georg-pinzel...,https://www.wikiart.org/en/johann-georg-pinzel,johann-georg-pinzel_samson-tearing-the-lion-s-...,0
4111,57728337edc2cb3880fa19a7,Minimalism,Richard Tuttle,Fountain,1965,https://uploads7.wikiart.org/images/richard-tu...,https://www.wikiart.org/en/richard-tuttle/foun...,https://www.wikiart.org/en/richard-tuttle,richard-tuttle_fountain-1965.jpg,0


Taking artist name

In [11]:
unretrieved['artist'] = unretrieved['name'].map(lambda x: x.split('_')[0])
unretrieved['artist_1'] = unretrieved['Artist'].map(lambda x: '-'.join(x.lower().split(' ')))

C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_31172\1062937287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved['artist'] = unretrieved['name'].map(lambda x: x.split('_')[0])
C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_31172\1062937287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved['artist_1'] = unretrieved['Artist'].map(lambda x: '-'.join(x.lower().split(' ')))


In [12]:
unretrieved.drop(['ID', 'Category', 'Artist', 'Year', 'Painting Info URL', 'Artist Info URL', 'name_in_artgraph','Image URL'],
                 axis = 1, inplace=True)

C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_31172\2823596078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved.drop(['ID', 'Category', 'Artist', 'Year', 'Painting Info URL', 'Artist Info URL', 'name_in_artgraph'],


,Title,Image URL,name,artist,artist_1
2,Uncle Piacsek in front of the Black Sideboard,https://uploads3.wikiart.org/images/j-zsef-rip...,j-zsef-rippl-r-nai_uncle-piacsek-in-front-of-t...,j-zsef-rippl-r-nai,jozsef-rippl-ronai
3,Monk. For the Play &#39;Mazeppa&#39;,https://uploads2.wikiart.org/00124/images/vady...,vadym-meller_monk-for-the-play-mazeppa-1920.jpg,vadym-meller,vadym-meller
4,The Defence of Sarago&#231;a,https://uploads6.wikiart.org/images/david-wilk...,david-wilkie_the-defence-of-sarago-a.jpg,david-wilkie,david-wilkie
8,Mirror and Crushed Shells,https://uploads7.wikiart.org/images/robert-smi...,robert-smithson_mirror-and-crushed-shells-1969...,robert-smithson,robert-smithson
9,One Ton Prop (House of Cards),https://uploads7.wikiart.org/images/richard-se...,richard-serra_one-ton-prop-house-of-cards-1969...,richard-serra,richard-serra
...,...,...,...,...,...
4105,In the Hold,https://uploads0.wikiart.org/images/david-bomb...,david-bomberg_in-the-hold-1914.jpg,david-bomberg,david-bomberg
4108,Allstar Galilee,https://uploads6.wikiart.org/images/stephen-mu...,stephen-mueller_allstar-galilee-1970.jpg,stephen-mueller,stephen-mueller
4109,Samson tearing the lion&#39;s mouth,https://uploads5.wikiart.org/images/johann-geo...,johann-georg-pinzel_samson-tearing-the-lion-s-...,johann-georg-pinzel,johann-georg-pinzel
4111,Fountain,https://uploads7.wikiart.org/images/richard-tu...,richard-tuttle_fountain-1965.jpg,richard-tuttle,richard-tuttle


In [16]:
# function that gets all the paintings of a specific artist
def get_paintings_by_artist(artist_name):
    base_query = 'https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl={artist}&json=2'
    return requests.get(base_query.format(artist=artist_name)).json()

# function that gets all painting infos for a specific artwork
def get_painting_info(painting_id: int):
    base_query = 'https://www.wikiart.org/en/App/Painting/ImageJson/{painting}'
    return requests.get(base_query.format(painting=painting_id)).json()

# function that gets all url unmatched artworks with respect to wikiart artwork info
def retrieve_artworks_by_url(artist_name, metadata):
    paintings = get_paintings_by_artist(artist_name)
    paintings = map(lambda x: x['contentId'], paintings)
    paintings = map(lambda x: f"{get_painting_info(x)['url']}.jpg", paintings)
    return metadata[metadata['name'].isin(paintings)].name.tolist()

# function that gets all unmatched artworks with respect to basic info got by searching just paintings of spceific artwork
def retrieve_artworks(artist_name, metadata):
    paintings = get_paintings_by_artist(artist_name)
    titles = list(map(lambda x: x['title'], paintings))
    names = list(map(lambda x: x['image'].split('/')[-1][:-10], paintings))
    return metadata[(metadata['Title'].isin(titles)) | (metadata['name'].isin(names))].name.tolist()

Group paintings by artist

In [17]:
unretrieved['name'] = unretrieved['name'].map(lambda x: x.split('_')[1])
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')

C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_31172\4122269952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved['name'] = unretrieved['name'].map(lambda x: x.split('_')[1])


,artist,artist_1,metadata
0,aaron-siskind,aaron-siskind,"[[Feet 102, feet-102-1957.jpg], [Jerome, Arizo..."
1,abraham-manievich,abraham-manievich,"[[Destruction of the Ghetto, Kiev, destruction..."
2,ad-reinhardt,ad-reinhardt,"[[Study for a Painting, study-for-a-painting-1..."
3,adolph-menzel,adolph-menzel,"[[Afternoon in the Tuileries Gardens, afternoo..."
4,adriaen-brouwer,adriaen-brouwer,"[[Youth Making a Face, youth-making-a-face.jpg..."
...,...,...,...
496,wolfgang-paalen,wolfgang-paalen,"[[Genius of the Species, genius-of-the-species..."
497,wyndham-lewis,wyndham-lewis,"[[The Crowd, the-crowd-1915.jpg]]"
498,xul-solar,xul-solar,"[[Reptil que sube, reptil-que-sube-1920.jpg], ..."
499,yannoulis-chalepas,yannoulis-chalepas,"[[Mercury, Venus and Pegasus, mercury-venus-an..."


Retrieve artworks following **1**.

In [19]:
retrieved = grouped.progress_apply(lambda x: retrieve_artworks(x.artist, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

100%|██████████| 501/501 [02:16<00:00,  3.67it/s]


0           [feet-102-1957.jpg, jerome-arizona-1949.jpg]
1              [destruction-of-the-ghetto-kiev-1919.jpg]
2                      [study-for-a-painting-1938-1.jpg]
3      [afternoon-in-the-tuileries-gardens-1867(1).jp...
4      [youth-making-a-face.jpg, fumatore.jpg, the-bi...
                             ...                        
496                     [genius-of-the-species-1938.jpg]
497                                 [the-crowd-1915.jpg]
498    [reptil-que-sube-1920.jpg, pegaso-de-sol-1922....
499    [mercury-venus-and-pegasus-1933.jpg, satyr-pla...
500                    [disasters-of-the-war-1926-2.jpg]
Length: 501, dtype: object

In [21]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')

,artist,artist_1,names
0,aaron-siskind,aaron-siskind,"[feet-102-1957.jpg, jerome-arizona-1949.jpg]"
1,abraham-manievich,abraham-manievich,[destruction-of-the-ghetto-kiev-1919.jpg]
2,ad-reinhardt,ad-reinhardt,[study-for-a-painting-1938-1.jpg]
3,adolph-menzel,adolph-menzel,[afternoon-in-the-tuileries-gardens-1867(1).jp...
4,adriaen-brouwer,adriaen-brouwer,"[youth-making-a-face.jpg, fumatore.jpg, the-bi..."
...,...,...,...
496,wolfgang-paalen,wolfgang-paalen,[genius-of-the-species-1938.jpg]
497,wyndham-lewis,wyndham-lewis,[the-crowd-1915.jpg]
498,xul-solar,xul-solar,"[reptil-que-sube-1920.jpg, pegaso-de-sol-1922...."
499,yannoulis-chalepas,yannoulis-chalepas,"[mercury-venus-and-pegasus-1933.jpg, satyr-pla..."


placing the first retrieved artwroks

In [23]:
total['ret'] = retrieved

Making comparison and underline unretrieved artworks

In [25]:
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)
total

,artist,artist_1,names,ret,difference
0,aaron-siskind,aaron-siskind,"[feet-102-1957.jpg, jerome-arizona-1949.jpg]","[feet-102-1957.jpg, jerome-arizona-1949.jpg]",[]
1,abraham-manievich,abraham-manievich,[destruction-of-the-ghetto-kiev-1919.jpg],[destruction-of-the-ghetto-kiev-1919.jpg],[]
2,ad-reinhardt,ad-reinhardt,[study-for-a-painting-1938-1.jpg],[study-for-a-painting-1938-1.jpg],[]
3,adolph-menzel,adolph-menzel,[afternoon-in-the-tuileries-gardens-1867(1).jp...,[afternoon-in-the-tuileries-gardens-1867(1).jp...,[]
4,adriaen-brouwer,adriaen-brouwer,"[youth-making-a-face.jpg, fumatore.jpg, the-bi...","[youth-making-a-face.jpg, fumatore.jpg, the-bi...",[]
...,...,...,...,...,...
496,wolfgang-paalen,wolfgang-paalen,[genius-of-the-species-1938.jpg],[genius-of-the-species-1938.jpg],[]
497,wyndham-lewis,wyndham-lewis,[the-crowd-1915.jpg],[the-crowd-1915.jpg],[]
498,xul-solar,xul-solar,"[reptil-que-sube-1920.jpg, pegaso-de-sol-1922....","[reptil-que-sube-1920.jpg, pegaso-de-sol-1922....",[]
499,yannoulis-chalepas,yannoulis-chalepas,"[mercury-venus-and-pegasus-1933.jpg, satyr-pla...","[mercury-venus-and-pegasus-1933.jpg, satyr-pla...",[]


In [27]:
total.difference.map(len).sum()

53

Attualmente ce ne mancano 53

# STAGE 2: USARE L'ALTRO NOME

In [ ]:
import itertools
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))
unretrieved = unretrieved[unretrieved.name.str.isin(paintings_still_unretrieved)]

In [ ]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')
retrieved = grouped.progress_apply(lambda x: retrieve_artworks(x.artist, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

In [ ]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')
total['ret'] = retrieved
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)

In [ ]:
total